In [ ]:
import sqlalchemy
import pymysql
from urllib import parse


In [ ]:
import requests
import pandas as pd
from tqdm import tqdm


url = "https://tmacs.kotsa.or.kr/web/TG/TG200/TG2100S/Tgsearch01_AJAX.jsp"
payload = {"gubun": "S1181",
"mid": "S1181",
"sYear": "2022",
"eYear": "2022",
"sido": "11000",
"jijace": "11680",
"kind1": "s104",
"kind2": "s1041",
"cartype": "carall"}

year_list=["2018","2019","2020","2021","2022"]
jijace_list=[["강남구" , 11680] , ["강동구" ,11740], ["강북구" , 11305], ["강서구" , 11500],  ["관악구", 11620], ["광진구" , 11215], ["구로구" ,11530],
            ["금천구",11545], ["노원구" , 11350] ,["도봉구",11320], ["동대문구",11230], ["동작구" , 11590], ["마포구" ,11440] , ["서대문구" ,11410] , ["서초구", 11650],
            ["성동구" , 11200], ["성북구" , 11290], ["송파구" , 11710], ["양천구" ,11470], ["영등포구" , 11560], ["용산구" , 11170], ["은평구" , 11380],
            ["종로구", 11110] , ["중구" , 11140], ["중랑구",11260]]
total_df=pd.DataFrame()
for i in tqdm(range(len(jijace_list))):
    for j in range(len(year_list)):
        payload["sYear"]=year_list[j]
        payload["eYear"]=year_list[j]
        payload["jijace"]=jijace_list[i][1]
            # print(year_list[j])
            # print(jijace_list[i][1])
        r = requests.post(url, data=payload)
        bs = eval(r.text)
        df = pd.DataFrame(bs)
        df["YEAR"]=year_list[j]
        total_df = pd.concat([total_df, df], axis = 0)
            # pd.set_option('display.max_columns', None)

In [ ]:
import sqlalchemy
import pymysql
from urllib import parse

user = 
password =
host=
port = 3306
database = 
password = parse.quote_plus(password)
df = sqlalchemy.create_engine(f"mysql://{user}:{password}@{host}:{port}/{database}")
# df1 = pd.read_sql_query("select * from minini", con=df)
con =pymysql.connect(host='', user='', password='', db='', charset='utf8')
cur = con.cursor()


In [ ]:
total_df["BASE_CATEGORY"] = "운전경력"

In [ ]:
sql3 = """CREATE Table carexp_accident
(
    JIJACE_NM VARCHAR(30),
    CHISA FLOAT,
    ACC_HUM INT,
    SIDO_CD INT,
    GUBUN_NM VARCHAR(30),
    SLTWD_CNT INT,
    SERI_ACC INT,
    SIDO_NM VARCHAR(10),
    AC_MODE VARCHAR(4),
    CRT_JIJACE_CD INT ,
    BASE_NAME VARCHAR(30),
    BIG_CNT INT,
    PSGT_CNT INT,
    WND_ACC INT,
    ACC_CNT INT,
    TRK_CNT INT,
    DEATH_CNT INT,
    BASE_CODE VARCHAR(10),
    WND_CNT INT,
    DEATH_ACC INT,
    GUBUN VARCHAR(5),
    SERI_CNT INT,
    SLTWD_ACC INT,
    YEAR INT,
    BASE_CATEGORY VARCHAR(10)
)"""

In [ ]:
cur.execute(sql3)
con.commit()

In [ ]:
sql3 ="INSERT  INTO carexp_accident VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s ,%s ,%s ,%s)"

In [ ]:
total_df.apply(lambda x : cur.execute(sql3, x.tolist() ), axis=1)

In [ ]:
con.commit()

In [ ]:
import pickle
with open("./carexp_accident.pkl", "wb") as f:
    pickle.dump(total_df, f)

In [ ]:
import folium
import json
map = folium.Map(location=[37.5502, 126.982], zoom_start=11, tiles="cartodb positron")
geo_str = requests.get("https://t1.daumcdn.net/cfile/tistory/272C224C58B4BD540B").json()
data = total_df.groupby(['JIJACE_NM'],as_index=False)[['ACC_HUM']].sum()
m = folium.Choropleth(geo_str, 
                  data = data,
                  columns=['JIJACE_NM', 'ACC_HUM'],
                  fill_color="PuRd",
                  key_on="feature.id").add_to(map)

In [ ]:
map